In [1]:
import pandas as pd
import pickle
import os
from multiprocessing import Manager, Pool
from operator import mul
from functools import reduce
from math import log
# from transformers import AutoTokenizer
import gc
import sys
from tqdm import tqdm
import math

In [2]:
print('main process PID: ', os.getpid())

ngram_range = [2, 3, 4, 5, 6, 7, 8, 9, 10] 
min_occurrences = 101
pool_size = 8
pmi_data_folder = './pmi/pmi_data' # updated to anonymize

main process PID:  31170


In [43]:
print(freq_log_prob_dict['AT'])

-3.248176839782861


In [3]:
manager = Manager()
lock = manager.Lock()
pmi = manager.dict()

print('loading ngram_frequency file')
df = pd.read_pickle(f'{pmi_data_folder}/ngram_frequencies.pkl')
print('load done, next transfer df to dict and move it to manager')

print(df.head())

gram_dict = manager.dict(df.set_index('ngram').to_dict()['frequency'])
print('Transfer over. Prepare to calculate PMI scores')

loading ngram_frequency file
load done, next transfer df to dict and move it to manager
  ngram  frequency
0     N   20316135
1     T  199065398
2     A  198550581
3     C  135606706
4     G  135906690
Transfer over. Prepare to calculate PMI scores


In [4]:
for ngram in gram_dict.keys():
    if gram_dict[ngram] < 101 or 'N' in ngram:
        gram_dict.pop(ngram)

In [5]:
print(gram_dict['A'])

198550581


In [6]:
from collections import defaultdict
discounting_factor= defaultdict(float)

In [7]:
for ngram in tqdm(gram_dict.keys()):
    discounting_factor[ngram]= math.log(gram_dict[ngram])/(math.log(gram_dict[ngram])+math.log(101))

100%|██████████| 1049598/1049598 [01:16<00:00, 13678.17it/s]


In [8]:
print(discounting_factor['A'], discounting_factor['TCGCGA'])

0.8054471058684937 0.6338270601245872


In [9]:
print(gram_dict['ATA'], gram_dict['TCGCGA'])

13926219 2947


In [10]:
pmi_scores_data= pd.read_csv('pmi_scores_dp_final.csv') # name changed
pmi_scores_data.head()

,ngram,naive-pmi,true-pmi,dp-true-pmi
0,ACACACACAC,11.802667,1.825503,8.296317
1,GTGTGTGTGT,11.793167,2.094753,8.295802
2,GCACTCCAGC,11.723549,2.204254,8.231116
3,GCTGGAGTGC,11.717789,2.220593,8.227502
4,CCTCAGCCTC,12.117446,2.178819,8.095604


In [11]:
print(pmi_scores_data.shape)

(1048575, 4)


In [12]:
norm_dp_true_pmi= []

for row_id in tqdm(range(pmi_scores_data.shape[0])):
    ngram_seq= pmi_scores_data.iloc[row_id, 0]
    norm_score= pmi_scores_data.iloc[row_id, 3]* discounting_factor[ngram_seq]
    norm_dp_true_pmi.append(norm_score)

100%|██████████| 1048575/1048575 [00:53<00:00, 19777.63it/s]


In [13]:
print(len(norm_dp_true_pmi))

1048575


In [14]:
pmi_scores_data= pmi_scores_data.assign(norm_dp_true_pmi= norm_dp_true_pmi)
pmi_scores_data.head()

,ngram,naive-pmi,true-pmi,dp-true-pmi,norm_dp_true_pmi
0,ACACACACAC,11.802667,1.825503,8.296317,5.964404
1,GTGTGTGTGT,11.793167,2.094753,8.295802,5.966038
2,GCACTCCAGC,11.723549,2.204254,8.231116,5.793672
3,GCTGGAGTGC,11.717789,2.220593,8.227502,5.791656
4,CCTCAGCCTC,12.117446,2.178819,8.095604,5.757429


In [15]:
pmi_scores_data.to_csv('norm_pmi_scores_dp_final.csv', index=False) # name updated to anonymize

In [1]:
import pandas as pd
import pickle

In [2]:
pmi_scores_data= pd.read_csv('final_vocab_2ndAugust_100K.csv')
pmi_scores_data.head()

,ngram,naive.pmi,true.pmi,dp.true.pmi,norm_dp_true_pmi,ngram_size,naive.pmi.rank,dp.true.pmi.rank,norm_dp_true_pmi_rank,adv_pmi_rank_drop,norm_adv_pmi_rank_drop
0,GTGTGTGTGT,11.793167,2.094753,8.295802,5.966038,10,19,2,1,-17,-1
1,ACACACACAC,11.802667,1.825503,8.296317,5.964404,10,14,1,2,-13,1
2,GCACTCCAGC,11.723549,2.204254,8.231116,5.793672,10,27,3,3,-24,0
3,GCTGGAGTGC,11.717789,2.220593,8.227502,5.791656,10,29,4,4,-25,0
4,CCTCAGCCTC,12.117446,2.178819,8.095604,5.757429,10,5,5,5,0,0


In [3]:
pmi_scores_data6= pmi_scores_data[pmi_scores_data['ngram_size']==6]
pmi_scores_data6= pmi_scores_data6.sort_values(by=['norm_dp_true_pmi'], ascending=False)
pmi_scores_data6.head()

,ngram,naive.pmi,true.pmi,dp.true.pmi,norm_dp_true_pmi,ngram_size,naive.pmi.rank,dp.true.pmi.rank,norm_dp_true_pmi_rank,adv_pmi_rank_drop,norm_adv_pmi_rank_drop
47293,CCTCCC,5.346571,1.517800,2.849146,2.103076,6,71560,63232,47294,-8328,-15938
47960,GGGAGG,5.334066,1.399038,2.835727,2.093004,6,71843,63382,47961,-8461,-15421
51425,GTGTGT,4.519335,1.177967,2.770652,2.042932,6,92138,64077,51426,-28061,-12651
52687,CCCAGC,5.248139,1.364859,2.751367,2.026842,6,73910,64321,52688,-9589,-11633
52930,GCTGGG,5.241092,1.173914,2.746890,2.023631,6,74109,64373,52931,-9736,-11442


In [4]:
pmi_scores_data6= pmi_scores_data6.assign(norm_pmi_rank_rel= range(1, (pmi_scores_data6.shape[0]+1)))
pmi_scores_data6.head()

,ngram,naive.pmi,true.pmi,dp.true.pmi,norm_dp_true_pmi,ngram_size,naive.pmi.rank,dp.true.pmi.rank,norm_dp_true_pmi_rank,adv_pmi_rank_drop,norm_adv_pmi_rank_drop,norm_pmi_rank_rel
47293,CCTCCC,5.346571,1.517800,2.849146,2.103076,6,71560,63232,47294,-8328,-15938,1
47960,GGGAGG,5.334066,1.399038,2.835727,2.093004,6,71843,63382,47961,-8461,-15421,2
51425,GTGTGT,4.519335,1.177967,2.770652,2.042932,6,92138,64077,51426,-28061,-12651,3
52687,CCCAGC,5.248139,1.364859,2.751367,2.026842,6,73910,64321,52688,-9589,-11633,4
52930,GCTGGG,5.241092,1.173914,2.746890,2.023631,6,74109,64373,52931,-9736,-11442,5


In [5]:
print(pmi_scores_data6.shape)

(4096, 12)


In [6]:
pmi_scores_data6.to_csv('final_6mer_pmiscore_ranking_16thSept.csv', index=False)

In [19]:
from collections import defaultdict

norm_pmi_rank_dict= defaultdict(int)

for row_id in range(pmi_scores_data6.shape[0]):
    norm_pmi_rank_dict[pmi_scores_data6.iloc[row_id, 0]]= pmi_scores_data6.iloc[row_id, 11]

In [23]:
#print(norm_pmi_rank_dict)
with open('./pmi/pmi_data/seq_to_pmi_rank.pkl', "wb") as file: # path and filename updated to anonymize
    pickle.dump(norm_pmi_rank_dict, file, protocol=pickle.HIGHEST_PROTOCOL)